In [12]:
from ultralytics.data.converter import convert_coco
convert_coco(labels_dir="dataset/yolo_custom/labels/", use_segments=True)

Annotations D:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\labels\instances_train.json: 100%|██████████| 80/80 [00:00<00:00, 1604.02it/s]
Annotations D:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\labels\instances_val.json: 100%|██████████| 20/20 [00:00<00:00, 1667.91it/s]

COCO data converted successfully.
Results saved to D:\PG_Gdansk\Torun_secondment\Experiments\coco_converted2


In [3]:
from ultralytics.data.converter import convert_segment_masks_to_yolo_seg

# For datasets like COCO with 80 classes
convert_segment_masks_to_yolo_seg(masks_dir="dataset/coco/annotations/", output_dir="dataset", classes=3)

In [ ]:
import os, json, random, shutil
from pathlib import Path

# Paths
dataset_dir = Path("dataset/coco")
images_dir = dataset_dir / "images"
all_images = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))

# Load COCO annotations
with open(dataset_dir / "annotations//instances_default.json") as f:
    coco = json.load(f)

# Split images
random.seed(42)
random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))
train_imgs, val_imgs = all_images[:split_idx], all_images[split_idx:]

def filter_annotations(imgs, split_name):
    img_ids = {img.stem for img in imgs}
    images = [im for im in coco["images"] if Path(im["file_name"]).stem in img_ids]
    img_id_set = {im["id"] for im in images}
    annotations = [ann for ann in coco["annotations"] if ann["image_id"] in img_id_set]

    split = {
        "images": images,
        "annotations": annotations,
        "categories": coco["categories"],
    }
    with open(dataset_dir / f"annotations/instances_{split_name}.json", "w") as f:
        json.dump(split, f)

# Save splits
filter_annotations(train_imgs, "train")
filter_annotations(val_imgs, "val")

# Move images
(train_dir := dataset_dir / "images/train").mkdir(parents=True, exist_ok=True)
(val_dir := dataset_dir / "images/val").mkdir(parents=True, exist_ok=True)

for img in train_imgs:
    shutil.copy(img, train_dir / img.name)
for img in val_imgs:
    shutil.copy(img, val_dir / img.name)

print(f"Train: {len(train_imgs)} images, Val: {len(val_imgs)} images")


Train: 80 images, Val: 20 images


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="data.yaml", epochs=100, imgsz=640)

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 segmentation model
model = YOLO("yolov8s-seg.pt")

# Train
model.train(
    data="data.yaml",
    epochs=100,
    imgsz=640,
    batch=4,
    task="segment"
)


In [1]:
from ultralytics import YOLO

# Load YOLO11 segmentation model
model = YOLO("yolo11n-seg.pt")

# Train it
model.train(
    data="data.yaml",
    epochs=100,
    imgsz=640,  # Reduce image size to avoid CUDA OOM
    task="segment"
)


New https://pypi.org/project/ultralytics/8.3.183 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.182  Python-3.8.20 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, opt

train: Scanning D:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\labels\train.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 2096.6311.0 MB/s, size: 547.7 KB)


val: Scanning D:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\labels\val.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to runs\segment\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.90it/s]


                   all         20         60      0.993          1      0.995      0.956      0.993          1      0.995      0.902
               Nucleus         20         20      0.992          1      0.995      0.927      0.992          1      0.995      0.901
                  lens         20         20      0.989          1      0.995      0.995      0.989          1      0.995      0.876
                cornea         20         20      0.997          1      0.995      0.946      0.997          1      0.995      0.928
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\segment\train4


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000027EB98C8790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    

In [2]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/segment/train4/weights/best.pt")

# Run inference
results = model.predict(source="dataset/yolo_custom/images/val/CN041_GS_28_F__OS__6-0D__2__Vertical.png", save=True, imgsz=640)



image 1/1 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN041_GS_28_F__OS__6-0D__2__Vertical.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 12.4ms
Speed: 4.6ms preprocess, 12.4ms inference, 18.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict6


In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load model
model = YOLO("runs/segment/train/weights/best.pt")

# Define class names and colors (BGR format for OpenCV)
class_names = ["Nucleus", "lens", "cornea"]
class_colors = {
    0: (255, 0, 0),   # Blue for class1
    1: (0, 255, 0),   # Green for class2
    2: (0, 0, 255)    # Red for class3
}

# Run inference
results = model("dataset/yolo_custom/images/val/CN041_GS_28_F__OS__6-0D__2__Vertical.png")

for r in results:
    img = r.orig_img.copy()
    masks = r.masks.xy  # list of masks (polygons)

    for mask, cls in zip(r.masks.xy, r.boxes.cls):
        cls = int(cls)
        color = class_colors[cls]
        # Convert polygon mask to int32 format
        pts = np.array(mask, dtype=np.int32)
        cv2.fillPoly(img, [pts], color)

    cv2.imwrite("runs/segment/predict/output_img.png", img)

    # cv2.imshow("Custom Segmentation", img)
    # cv2.waitKey(0)



image 1/1 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN041_GS_28_F__OS__6-0D__2__Vertical.png: 640x640 1 class1, 1 class2, 1 class3, 12.4ms
Speed: 4.2ms preprocess, 12.4ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)


In [3]:
results = model.predict(source="dataset/yolo_custom/images/val/", save=True, imgsz=640)



image 1/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN038_MM_37_F__OS__6-0D__4__Vertical.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 21.3ms
image 2/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN038_MM_37_F__OS__6-0D__6__Horizontal.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 27.9ms
image 3/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN039_AK_27_F__OD__0-6D__4__Vertical.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 27.7ms
image 4/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN039_AK_27_F__OD__0-6D__6__Vertical.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 15.8ms
image 5/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN039_AK_27_F__OD__6-0D__0__Horizontal.png: 640x640 1 Nucleus, 1 lens, 1 cornea, 21.4ms
image 6/20 d:\PG_Gdansk\Torun_secondment\Experiments\dataset\yolo_custom\images\val\CN039_AK_27_F__OD__6-0D__2__Vertical.png: 640x640 1 Nucleus, 

In [10]:
import cv2

# Load image
img = cv2.imread("dataset/yolo_custom/images/val/CN041_GS_28_F__OS__6-0D__2__Vertical.png")

# Check if loaded correctly
if img is None:
    print("Error: image not found!")
else:
    h, w, c = img.shape
    print(f"Width: {w}, Height: {h}, Channels: {c}")


Width: 1600, Height: 1600, Channels: 3


In [6]:
import cv2
import numpy as np

# Define colors for each class
class_colors = {
    0: (255, 0, 0),   # Blue
    1: (0, 255, 0),   # Green
    2: (0, 0, 255)    # Red
}

img_path = "dataset/yolo_custom/images/val/CN041_GS_28_F__OS__6-0D__2__Vertical.png"
label_path = "dataset/yolo_custom/labels/val/CN041_GS_28_F__OS__6-0D__2__Vertical.txt"

img = cv2.imread(img_path)
h, w = img.shape[:2]

with open(label_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        cls = int(parts[0])
        coords = np.array(parts[1:], dtype=np.float32).reshape(-1, 2)
        # Convert normalized coordinates to pixel coordinates
        pts = np.stack([coords[:, 0] * w, coords[:, 1] * h], axis=-1).astype(np.int32)
        cv2.fillPoly(img, [pts], class_colors[cls])

cv2.imshow("Segmentation Visualization", img)
cv2.waitKey(0)
cv2.destroyAllWindows()